In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# CONV_CHAT_BOT Setup

In [ ]:
%%capture
!pip install langchain
!pip install pypdf
!pip install sentence_transformers
!pip install chromadb
!pip install accelerate
!pip install --upgrade accelerate
!pip install bitsandbytes
!pip install onnxruntime

In [ ]:
from langchain.docstore.document import Document
from langchain.vectorstores import Chroma
import os
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings

# Load multiple documents
pdf_files = ["/content/drive/MyDrive/Books/latitude-15-5580-laptop_owners-manual_en-us.pdf"]

# Create an empty list to store documents
all_documents = []
# Load documents using PyPDFLoader
for pdf_file in pdf_files:
    loader = PyPDFLoader(pdf_file)
    documents = loader.load()
    all_documents.extend(documents)

all_documents.extend(documents)

# Split the documents
text_splitter = CharacterTextSplitter(chunk_size=400, chunk_overlap=40)
all_documents = text_splitter.split_documents(all_documents)

# Initialize HuggingFaceEmbeddings
hf_embed = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

# Create Chroma vector store from multiple documents
Vector_db = Chroma.from_documents(collection_name="document_docs", documents=all_documents, embedding=hf_embed, persist_directory="/content/drive/MyDrive/Manual_bot/Data/Docs/Persist_dir")

# Perform a similarity search (if needed)
# Vector_db.similarity_search("dummy")  # Tickle it to persist metadata (?)

# Persist the vector store
Vector_db.persist()


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.1k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

#1/ Download our 2 embeddings model from hugging face
(same as data preparation)

In [ ]:
def get_available_gpus():
    """Get a list of available gpu devices (formatted as strings)."""
    from tensorflow.python.client import device_lib
    local_device_protos = device_lib.list_local_devices()
    print(local_device_protos)
    return [x.name for x in local_device_protos if x.device_type == 'GPU']

In [ ]:
# Start here to load a previously-saved DB
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

if len(get_available_gpus()) == 0:
  Exception("Running dolly without GPU will be slow. We recommend you switch to a Single Node cluster with at least 1 GPU to properly run this demo.")

# gardening_vector_db_path = "/dbfs"+demo_path+"/vector_db"
vector_db_path = "/content/drive/MyDrive/Manual_bot/Data/Docs/db_path"

hf_embed = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
db = Chroma(collection_name="document_docs", embedding_function=hf_embed, persist_directory=vector_db_path)

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 6695946088697288518
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 13120897024
locality {
  bus_id: 1
  links {
  }
}
incarnation: 13118036708175737627
physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"
xla_global_id: 416903419
]


In [ ]:
conversation_history = []

In [ ]:
def get_similar_docs(question, similar_doc_count):
    similar_docs = Vector_db.similarity_search(question, similar_doc_count)

    unique_docs = []
    seen_doc_contents = set()

    for doc in similar_docs:
        doc_content = doc.page_content  # Assuming 'page_content' is the key for the document content
        if doc_content not in seen_doc_contents:
            unique_docs.append(doc)
            seen_doc_contents.add(doc_content)

    return unique_docs


# Let's test it with blackberries:
for doc in get_similar_docs("What is prison?", 4):
    print(doc)

page_content='Feature Specification\n51Whr\n68Whr\n92Whr\n42 Whr :\nDepth 181 mm (7.126 inches)\nHeight 7.05 mm (0.28 inch)\nWidth 95.9 mm (3.78 inches)\nWeight 210 g (0.46 lb)\nVoltage 11.4 V DC\n51 Whr :\nDepth 181 mm (7.126 inches)\nHeight 7.05 mm (0.28 inch)\nWidth 95.9 mm (3.78 inches)\nWeight 250 g (0.55 lb)\nVoltage 11.4 V DC\n68 Whr :\nDepth 233.00 mm (9.17 inches)\nHeight 7.5 mm (0.28 inch)\nWidth 95.90 mm (3.78 inches)\nWeight 340 g (0.74 lb)\nVoltage 7.6 V DC\n92Whr :\nDepth 332.00 mm (13.07 inches)\nHeight 7.7 mm (0.303 inches)\nWidth 96.0 mm (3.78 inches)\nWeight 450.00 g (0.99 lb)\nVoltage 11.4 V DC\nLife span 300 discharge per charge cycles\nTemperature\nrange\nOperating ●Charge: 0°C to 50°C (32°F to 158°F)\n●Discharge: 0°C to 70°C (32°F to 122°F)\n●Operating: 0°C to 35°C (32°F to 95°F)\nNon-operating - 20°C to 65°C (- 4°F to 149°F)\nCoin cell battery 3 V CR2032 lithium coin cell\nAC Adapter specifications\nFeature Specification\nType 65 W / 90 W\nInput voltage 100 V AC 

# 3/ Prompt engineering with langchain
Now we can compose with a language model and prompting strategy to make a langchain chain that answers questions.

In [ ]:
template = """Below is an instruction that describes a task. Write a response that appropriately completes the request.

  Instruction:
  You are a helpful manual assistant. Your job is to provide answers to customer questions.
  Use only information in the following paragraphs to answer the question at the end.
  Explain the answer with reference to these paragraphs.
  If you don't have the information in below paragraphs then give response "The required information is not mentioned in the manual".

  {context}

  Question: {question}

  Response:
  """

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch
from langchain import PromptTemplate
from langchain.llms import HuggingFacePipeline
from langchain.chains.question_answering import load_qa_chain

def build_qa_chain():
  torch.cuda.empty_cache()
  model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0" # can use dolly-v2-3b or dolly-v2-7b for smaller model and faster inferences.

  # Increase max_new_tokens for a longer response
  # Other settings might give better results! Play around
  instruct_pipeline = pipeline(model=model_name, torch_dtype=torch.float16, trust_remote_code=True, device_map="auto", return_full_text=True, do_sample=True, max_new_tokens=512, top_p=0.95, top_k=50,model_kwargs={'load_in_8bit': True})
  # Note: if you use dolly 12B or smaller model but a GPU with less than 24GB RAM, use 8bit. This requires %pip install bitsandbytes
  # instruct_pipeline = pipeline(model=model_name, trust_remote_code=True, device_map="auto", model_kwargs={'load_in_8bit': True})
  # For GPUs without bfloat16 support, like the T4 or V100, use torch_dtype=torch.float16 below
  model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", torch_dtype=torch.float16, trust_remote_code=True)
  prompt = PromptTemplate(input_variables=['context', 'conversation_history', 'question'], template=template)

  hf_pipe = HuggingFacePipeline(pipeline=instruct_pipeline)
  # Set verbose=True to see the full prompt:
  return load_qa_chain(llm=hf_pipe, chain_type="stuff", prompt=prompt, verbose=True)

In [ ]:
qa_chain = build_qa_chain()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


#4/ Using the Chain for Simple Question Answering
That's it! It's ready to go. Define a function to answer a question and pretty-print the answer, w

In [ ]:
def displayHTML(html):
    """Display HTML in Jupyter notebook."""
    from IPython.display import HTML
    display(HTML(html))


def answer_question(question):
    similar_docs = get_similar_docs(question, similar_doc_count=4)
    conversation = "".join(conv["user_question"] + "\n" + conv["bot_response"] for conv in conversation_history)
    result = qa_chain.invoke({"input_documents": similar_docs, "conversation_history": conversation, "question": question})

    if result is None:
        result_html = f'<h1>Couldn\'t find data from the book</h1>'
        return

    conversation_history.append({"user_question": question, "bot_response": result["output_text"]})
    while len(conversation_history) != 1:
        conversation_history.pop(0)

    result_html = f"<p><blockquote style=\"font-size:24\">{question}</blockquote></p>"
    result_html += f"<p><blockquote style=\"font-size:18px\">{result['output_text']}</blockquote></p>"
    result_html += "<p><hr/></p>"
    for d in similar_docs:
        source_id = d.metadata["source"]
        page = d.metadata['page']
        result_html += f"<p><blockquote>{d.page_content}<br/>(Source: <p>{source_id} Page: {page}</p>)</blockquote></p>"
    displayHTML(result_html)


In [ ]:
conversation_history.clear()

In [ ]:
answer_question("What is the resolution of my screen?")



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Below is an instruction that describes a task. Write a response that appropriately completes the request.

  Instruction:
  You are a helpful manual assistant. Your job is to provide answers to customer questions.
  Use only information in the following paragraphs to answer the question at the end.
  Explain the answer with reference to these paragraphs.
  If you don't have the information in below paragraphs then give response "The required information is not mentioned in the manual".

  Video specifications
Feature Specification
Type Integrated on system board, hardware accelerated
Graphic cards i3, i5, i7 Intel HD Graphics 620 (dual core)
Intel HD Graphics 630 (quad core)
NVIDIA GeForce 930MX 64 Bit(dual core)
NVIDIA GeForce 940MX 64 Bit
Data bus Integrated video
External display
support●19-pin HDMI connector
●15-pin VGA connector
●DisplayPort over Type-C connector
Camera specif

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(



> Finished chain.

> Finished chain.


In [ ]:
answer_question("What is the battery life of my laptop?")



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Below is an instruction that describes a task. Write a response that appropriately completes the request.

  Instruction:
  You are a helpful manual assistant. Your job is to provide answers to customer questions.
  Use only information in the following paragraphs to answer the question at the end.
  Explain the answer with reference to these paragraphs.
  If you don't have the information in below paragraphs then give response "The required information is not mentioned in the manual".

  Technology and components
Topics:
• Power adapter
• Processors
• Chipsets
• Graphic options
• Display options
• Realtek ALC3246 Waves MaxxAudio Pro controller
• WLAN cards
• Hard drive options
• Camera features
• Memory features
• Realtek HD audio drivers
• Thunderbolt over USB Type-C
Power adapter
This laptop is shipped with 65 W or 90 W power adapter.
itself, and then pull firmly but gently to a

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(



> Finished chain.

> Finished chain.


In [ ]:
answer_question("What is GPU?")



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Below is an instruction that describes a task. Write a response that appropriately completes the request.

  Instruction:
  You are a helpful manual assistant. Your job is to provide answers to customer questions.
  Use only information in the following paragraphs to answer the question at the end.
  Explain the answer with reference to these paragraphs.
  If you don't have the information in below paragraphs then give response "The required information is not mentioned in the manual".

  Video specifications
Feature Specification
Type Integrated on system board, hardware accelerated
Graphic cards i3, i5, i7 Intel HD Graphics 620 (dual core)
Intel HD Graphics 630 (quad core)
NVIDIA GeForce 930MX 64 Bit(dual core)
NVIDIA GeForce 940MX 64 Bit
Data bus Integrated video
External display
support●19-pin HDMI connector
●15-pin VGA connector
●DisplayPort over Type-C connector
Camera specif

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(



> Finished chain.

> Finished chain.


In [ ]:
answer_question("How much GPU does my laptop have?")



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Below is an instruction that describes a task. Write a response that appropriately completes the request.

  Instruction:
  You are a helpful manual assistant. Your job is to provide answers to customer questions.
  Use only information in the following paragraphs to answer the question at the end.
  Explain the answer with reference to these paragraphs.
  If you don't have the information in below paragraphs then give response "The required information is not mentioned in the manual".

  Technology and components
Topics:
• Power adapter
• Processors
• Chipsets
• Graphic options
• Display options
• Realtek ALC3246 Waves MaxxAudio Pro controller
• WLAN cards
• Hard drive options
• Camera features
• Memory features
• Realtek HD audio drivers
• Thunderbolt over USB Type-C
Power adapter
This laptop is shipped with 65 W or 90 W power adapter.
itself, and then pull firmly but gently to a

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(



> Finished chain.

> Finished chain.


In [ ]:
answer_question("How much ram can my laptop support?")



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Below is an instruction that describes a task. Write a response that appropriately completes the request.

  Instruction:
  You are a helpful manual assistant. Your job is to provide answers to customer questions.
  Use only information in the following paragraphs to answer the question at the end.
  Explain the answer with reference to these paragraphs.
  If you don't have the information in below paragraphs then give response "The required information is not mentioned in the manual".

  2.Select Camera  from the apps list.
3.If the Camera  App is not available in the apps list, search for it.
Memory features
This laptop supports a minimum memory of :
●4 GB and a maximum of 32 GB DDR4 memory, up to 2133 MHz (dual core).
●4 GB and a maximum of 32 GB DDR4 memory, up to 2400 MHz (quad core).
NOTE:  The memory module in the Dual Core processor will have 2400 MHz printed, but it perfor

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(



> Finished chain.

> Finished chain.


In [ ]:
answer_question("What is the contact support number if I get into trouble?")



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Below is an instruction that describes a task. Write a response that appropriately completes the request.

  Instruction:
  You are a helpful manual assistant. Your job is to provide answers to customer questions.
  Use only information in the following paragraphs to answer the question at the end.
  Explain the answer with reference to these paragraphs.
  If you don't have the information in below paragraphs then give response "The required information is not mentioned in the manual".

  Contacting Dell
NOTE:  If you do not have an active Internet connection, you can find contact information on your purchase invoice, packing
slip, bill, or Dell product catalog.
Dell provides several online and telephone-based support and service options. Availability varies by country and product, and
some services may not be available in your area. To contact Dell for sales, technical support, or

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(



> Finished chain.

> Finished chain.
